## Feature Engineering Amacı ve Stratejisi

Bu aşamanın temel amacı, Baseline modelin çözemediği doğrusal olmayan ilişkileri (saat, kategori, başlık içeriği) çözmek için modelin anlayacağı **yeni sayısal özellikler** (features) türetmektir. Türetilen bu yeni özellikler, R2 skorunu Baseline'ın (**0.6509**) üzerine çıkarmayı hedeflemektedir. Model seçimimiz, Baseline kıyaslaması sonucunda daha başarılı olduğu kanıtlanan **ağaç tabanlı (tree-based)** modellere odaklanacaktır.

## Adım 1: Veri Yükleme ve Zamanlama Özellikleri
Bu kod bloğu, veriyi yükler, gerekli temizliği yapar ve en temel özellik setimiz olan Zamanlama Özelliklerini (Ne zaman yayınlanmalı?) türetir.

In [4]:
import pandas as pd
import numpy as np

# 1. VERİYİ YÜKLEME VE TEMİZLİK
try:
    df = pd.read_csv('../data/US_youtube_trending_data.csv', encoding='utf-8')
except Exception:
    df = pd.read_csv('../data/US_youtube_trending_data.csv', encoding='latin1')

# Log dönüşümü için 0 olan değerleri temizliyoruz
df = df[(df['view_count'] > 0) & (df['likes'] > 0)].copy()

# 2. ZAMANLAMA ÖZELLİKLERİNİ TÜRETME
df['publishedAt'] = pd.to_datetime(df['publishedAt'])

# A) Yayın Saati (publish_hour)
df['publish_hour'] = df['publishedAt'].dt.hour 

# B) Yayın Günü (publish_day)
df['publish_day'] = df['publishedAt'].dt.day_name() 

# C) Haftasonu mu? (Boolean/Sayısal)
df['is_weekend'] = df['publishedAt'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

print("✅ Zamanlama Özellikleri Türetildi.")
print(f"Yeni Sütunlar: publish_hour, publish_day, is_weekend")

✅ Zamanlama Özellikleri Türetildi.
Yeni Sütunlar: publish_hour, publish_day, is_weekend


### Kategorik Özellik Haritalaması (JSON Mapping)
Kullanılan veri setinde video kategorileri sayısal ID'ler (Örn: 24, 10) olarak yer almaktadır. Bu sayısal ID'leri analizlerde anlamlılık katması için dışarıdan okunan JSON dosyası ile eşleştirilerek okunabilir kategori isimlerine (`category_name` sütununa) çevrilmiştir. Bu, modelin "Hangi kategoride yayınlanmalı?" sorusunu çözerken, sayılar yerine anlamlı etiketleri kullanmasını sağlar.

In [7]:
# --- EK KATEGORİ HARİTALAMA VE EŞLEŞTİRME ---
import json

# Kategori İsimlerini JSON dosyasından çekip eşleştiriyoruz
id_to_category = {}
try:
    with open('../data/US_category_id.json', 'r') as f:
        data = json.load(f)
        for category in data['items']:
            id_to_category[int(category['id'])] = category['snippet']['title']
    
    # 'categoryId' sütununu isimlerle değiştiriyoruz
    df['category_name'] = df['categoryId'].map(id_to_category)
    print("✅ Kategori İsimleri Haritalandı.")
except FileNotFoundError:
    print("⚠️ UYARI: JSON dosyası bulunamadı, kategoriler isme çevrilemedi.")
except KeyError as e:
    # Bu hata, categoryId sütununun adının farklı olmasından kaynaklanabilir.
    print(f"⚠️ UYARI: Kategori haritalaması başarısız oldu: {e}")

✅ Kategori İsimleri Haritalandı.


## Adım 2: Metin ve Etiket Özellikleri
Bu adım, modelin "Ne tür içerik trend oluyor?" sorusunu çözmesini sağlayacak.

In [8]:
# --- 3. METİN VE ETİKET ÖZELLİKLERİNİ TÜRETME ---

# A) Başlık Uzunluğu (Title Length)
df['title_length'] = df['title'].str.len()

# B) Etiket Sayısı (Tag Count)
# Tags sütunu string olduğu için '|' karakterine göre ayırıp sayıyoruz.
df['tag_count'] = df['tags'].apply(lambda x: 0 if x == '[none]' else len(x.split('|')))

# C) Noktalama İşaretleri (Başlık Psikolojisi)
# EDA'da görmüştük: Ünlem ve soru işareti izlenmeyi etkiliyor.
df['has_exclamation'] = df['title'].apply(lambda x: 1 if '!' in str(x) else 0)
df['has_question'] = df['title'].apply(lambda x: 1 if '?' in str(x) else 0)

print("✅ Metin ve Etiket Özellikleri Türetildi.")
print(f"Yeni Sütunlar: title_length, tag_count, has_exclamation, has_question")

✅ Metin ve Etiket Özellikleri Türetildi.
Yeni Sütunlar: title_length, tag_count, has_exclamation, has_question


###  Türetilmiş Özellikler Raporu (Temporal & Textual)

**1. Zamanlama Özellikleri:** `publish_hour`, `publish_day` ve `is_weekend` gibi özellikler türetilerek Baseline modelin çözemediği **zamana bağlı trend döngüsü** analizi için hazırlanmıştır.

**2. Metin ve Etiket Özellikleri:** Başlık uzunluğu, etiket sayısı ve noktalama işaretleri gibi özellikler türetilerek, **içerik türünün ve başlık stratejisinin** izlenme üzerindeki doğrusal olmayan etkileri ölçülmeye hazır hale getirilmiştir.

## Adım 3: Kategorik Kodlama (One-Hot Encoding)

In [15]:
# --- KATEGORİK KODLAMA (ONE-HOT ENCODING) ve SÜTUNLARI TEMİZLEME ---

# Kategorik olarak kullanacağımız sütunlar
categorical_features = ['category_name', 'publish_day'] 

# One-Hot Encoding uygulama
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

# Fazla detayı olan ve model için uygun olmayan ham sütunları atıyoruz
cols_to_drop = [
    'video_id', 
    'title', 
    'channelTitle', 
    'tags', 
    'publishedAt', 
    'trending_date', 
    'description', 
    'channelId',
    'thumbnail_link'
]

df_final = df_encoded.drop(columns=cols_to_drop, errors='ignore')

print("✅ Kategorik Kodlama Başarılı.")
print(f"Yeni Toplam Özellik Sayısı: {df_final.shape[1]} adet")

# BU SATIR, 35+ SÜTUNUN TAM LİSTESİNİ ÇIKTIDA GÖSTERİR
print("\n--- SON ÖZELLİK LİSTESİ (MODEL GİRDİSİ) ---")
print(df_final.columns.tolist())

✅ Kategorik Kodlama Başarılı.
Yeni Toplam Özellik Sayısı: 33 adet

--- SON ÖZELLİK LİSTESİ (MODEL GİRDİSİ) ---
['categoryId', 'view_count', 'likes', 'dislikes', 'comment_count', 'comments_disabled', 'ratings_disabled', 'publish_hour', 'is_weekend', 'title_length', 'tag_count', 'has_exclamation', 'has_question', 'category_name_Comedy', 'category_name_Education', 'category_name_Entertainment', 'category_name_Film & Animation', 'category_name_Gaming', 'category_name_Howto & Style', 'category_name_Music', 'category_name_News & Politics', 'category_name_Nonprofits & Activism', 'category_name_People & Blogs', 'category_name_Pets & Animals', 'category_name_Science & Technology', 'category_name_Sports', 'category_name_Travel & Events', 'publish_day_Monday', 'publish_day_Saturday', 'publish_day_Sunday', 'publish_day_Thursday', 'publish_day_Tuesday', 'publish_day_Wednesday']


###  Final Feature Set Raporu

**1. Başarı:** Ham sayısal özellik sayısı 2 iken, Zaman, Metin ve Kategorik analizler sonucunda toplam **33 türetilmiş özellik** içeren final bir set oluşturulmuştur. Bu, modelin Baseline modelde çözemediği **%35.84'lük varyansı** çözmek için gerekli tüm bilgileri içerdiğini gösterir.

**2. Amaç:** Bu yeni özellik seti, Baseline skorumuz olan **0.6509**'un üzerine çıkmak için hazırlanmıştır. Model, artık sadece beğeniye değil, aynı zamanda 33 farklı zamansal, metinsel ve kategorik faktöre bakarak tahmin yapacaktır.

**3. Karar:** Özellik sayımız ($N=33$) yüksek olduğu için, modelin hızını ve doğruluğunu artırmak amacıyla eğitime geçmeden önce **Feature Selection (Özellik Seçimi)** uygulanacaktır. Veri setimiz artık **ağaç tabanlı (XGBoost/Random Forest)** modellerle eğitime tamamen hazırdır.

## Adım 4: Feature Selection (Özellik Seçimi)


In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split 

# --- 1. VERİ AYARLAMA ---
# Hedef değişkeni (Y) logaritma ile hazırlıyoruz
y_target = np.log1p(df_final['view_count']) 

# Özellikleri (X) ayırıyoruz ('view_count' hedef olduğu için çıkarıldı)
X_features = df_final.drop(columns=['view_count']) 

# Veriyi bölüyoruz (Sadece SelectFromModel için train/test gerekli)
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_target, test_size=0.2, random_state=42
)

# 2. RANDOM FOREST İLE ÖNEM SIRASINI BELİRLEME
# RFR, her özelliğin modele ne kadar katkı sağladığını ölçer
model_selector = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_selector.fit(X_train, y_train)

# 3. ÖNEMLİ ÖZELLİKLERİ SEÇME
# Eşiği, ortalama önem değerinin üstüne koyarak seçimi daraltıyoruz.
# SelectFromModel, Random Forest'ın önem skorlarına göre otomatik seçim yapar.
sfm = SelectFromModel(model_selector, threshold='mean', prefit=True)

# Seçilen sütunları bulma
X_important = sfm.transform(X_features)
selected_features_mask = sfm.get_support()
selected_features = X_features.columns[selected_features_mask].tolist()

print("✅ Feature Selection Tamamlandı.")
print(f"Başlangıç Özellik Sayısı: {X_features.shape[1]}")
print(f"Seçilen Özellik Sayısı: {len(selected_features)}")
print("\n--- Seçilen En Önemli Özellikler ---")
print(selected_features)

c:\Users\fatmanur\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


✅ Feature Selection Tamamlandı.
Başlangıç Özellik Sayısı: 32
Seçilen Özellik Sayısı: 5

--- Seçilen En Önemli Özellikler ---
['likes', 'dislikes', 'comment_count', 'title_length', 'tag_count']


###  Nihai Feature Engineering Sonucu

**1. Özellik Azaltımı:** Başlangıçtaki 33 özellikten, Random Forest (RFR) algoritması kullanılarak en güçlü ve modele en çok katkı sağlayan **sadece 5 adet özellik** seçilmiştir.

**2. Seçilen Özellikler:** Modelin başarısının temelini **Etkileşim Metrikleri** (`likes`, `comment_count`, `dislikes`) oluşturmaktadır. Bunların yanına, en güçlü türetilmiş özellikler olan **`title_length`** ve **`tag_count`** eklenmiştir. Bu durum, model için **saat veya gün** gibi özelliklerin, temel etkileşim verileri kadar kritik olmadığını kanıtlamaktadır.

**3. Amaç:** Bu agresif azaltım, projenin final modelinin (XGBoost) eğitim süresini hızlandırmak ve alakasız 28 özelliğin modelin tahmin doğruluğunu düşürmesini engellemeyi amaçlar. **Final modelimiz, Baseline'ın 0.6509 R2 skorunu geçmek zorundadır.**